In [4]:
import os
import nibabel as nib
import numpy as np
from pathlib import Path

# Target shape
TARGET_SHAPE = (128, 128, 128)

def resize_volume(volume, target_shape):
    current_shape = volume.shape
    resized = np.zeros(target_shape, dtype=volume.dtype)

    crop = [min(c, t) for c, t in zip(current_shape, target_shape)]
    start_src = [(c - cr) // 2 for c, cr in zip(current_shape, crop)]
    start_dst = [(t - cr) // 2 for t, cr in zip(target_shape, crop)]

    src_slices = tuple(slice(start, start + size) for start, size in zip(start_src, crop))
    dst_slices = tuple(slice(start, start + size) for start, size in zip(start_dst, crop))

    resized[dst_slices] = volume[src_slices]
    return resized

def process_folder(input_folder, output_folder):
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    for subject in input_folder.iterdir():
        if subject.is_dir():
            out_subject = output_folder / subject.name
            out_subject.mkdir(exist_ok=True)
            for scan_type in ['PreT1', 'CeT1', 'Segmentation']:
                scan_path = subject / scan_type
                if scan_path.exists():
                    out_scan_path = out_subject / scan_type
                    out_scan_path.mkdir(exist_ok=True)
                    for file in scan_path.glob('*.nii.gz'):
                        img = nib.load(file)
                        data = img.get_fdata()
                        resized = resize_volume(data, TARGET_SHAPE)
                        out_path = out_scan_path / file.name.replace('.nii.gz', '_resized.nii.gz')
                        nib.save(nib.Nifti1Image(resized, img.affine, img.header), out_path)
                        print(f"✅ Saved: {out_path}")

# Process all splits
process_folder(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\normalized_dataset\train_norm', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized')
process_folder(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\normalized_dataset\val_norm', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\val_resized')
process_folder(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\normalized_dataset\test_norm', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\test_resized')


✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\PreT1\1_3p_left_breast_scout.nii_last_tp_norm_resized.nii.gz
✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\PreT1\1_t1-axial-locator_norm_resized.nii.gz
✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\PreT1\2_t1-axial-locator_norm_resized.nii.gz
✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\CeT1\31000_dynamic-3dfgre_ser_norm_resized.nii.gz
✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\CeT1\31001_dynamic-3dfgre_pe1_norm_resized.nii.gz
✅ Saved: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Resized_dataset\train_resized\ISPY1_1001\CeT1\3_dynamic-3dfgre.n

In [5]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
from pathlib import Path

def resize_volume(volume, target_shape=(128, 128, 128)):
    current_shape = volume.shape
    scale_factors = [t / c for t, c in zip(target_shape, current_shape)]
    return zoom(volume, scale_factors, order=0)  # Nearest-neighbor for masks

def binarize_mask(mask, threshold=0.5):
    return (mask > threshold).astype(np.uint8)

def process_masks(input_root, output_root, target_shape=(128, 128, 128)):
    input_root = Path(input_root)
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    for subject_dir in input_root.iterdir():
        if not subject_dir.is_dir():
            continue
        seg_dir = subject_dir / 'Segmentation'
        if not seg_dir.exists():
            continue
        for nii_file in seg_dir.glob('*.nii.gz'):
            img = nib.load(str(nii_file))
            data = img.get_fdata()
            resized = resize_volume(data, target_shape)
            binarized = binarize_mask(resized)

            out_dir = output_root / subject_dir.name
            out_dir.mkdir(parents=True, exist_ok=True)
            out_path = out_dir / nii_file.name.replace('.nii.gz', '_mask.nii.gz')
            nib.save(nib.Nifti1Image(binarized, img.affine), str(out_path))
            print(f"✅ Saved binarized mask: {out_path}")

# Process all sets
process_masks(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\dataset\train_clean', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks')
process_masks(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\dataset\train_clean', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\val_masks')
process_masks(r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\dataset\train_clean', r'C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\test_masks')


✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\32000_pe_segmentation_thresh70_mask.nii.gz
✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\32100_voi_pe_segmentation_thresh70_mask.nii.gz
✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\32101_voi_breast_tissue_segmentation_mask.nii.gz
✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\42000_pe_segmentation_thresh70_mask.nii.gz
✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\42001_breast_tissue_segmentation_mask.nii.gz
✅ Saved binarized mask: C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\Masks\train_masks\ISPY1_1001\42100_voi_pe_segmentation_thr